In [1]:
# implementation de bibliothèque 
import cv2
import os
import numpy as np

# Étape 1 : Charger les iamges

In [4]:
# acces a les images 
data_path = "dataSet/MVI_20012"

images = [f for f in os.listdir(data_path) if f.endswith(".jpg")]
images.sort()  # pour garde l oredre des frames 

print(f"{len(images)} images trouvées")

1549 images trouvées


# Etape 2 : Pre-traitement (Pre-processing)

In [11]:
data_path = "dataSet/MVI_20012"
preproc_path = "data_preproce_yolo"
os.makedirs(preproc_path, exist_ok=True)

images = sorted([f for f in os.listdir(data_path) if f.lower().endswith(".jpg")])

for i, img_name in enumerate(images):
    img_path = os.path.join(data_path, img_name)
    img = cv2.imread(img_path)

    # resize (optionnel, YOLO peut le faire tout seul)
    img = cv2.resize(img, (1280, 720))  # ou (640,480) si tu veux, mais attention à la perte de détails

    # Optionnel: légère amélioration contraste (CLAHE sur la luminance)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l2 = clahe.apply(l)
    lab2 = cv2.merge([l2, a, b])
    img2 = cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)

    save_path = os.path.join(preproc_path, f"frame_{i:04d}.jpg")
    cv2.imwrite(save_path, img2)

print("Pré-traitement YOLO terminé.")

Pré-traitement YOLO terminé.


# Etape 3: Charger YOLO et détecter les véhicules

In [44]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, trust_repo=True)
model.conf = 0.35  # seuil confiance global (ajuste 0.25 à 0.5)
model.iou = 0.45

vehicle_classes = {'car', 'bus', 'truck', 'motorcycle'}  # nom YOLOv5 = "motorcycle" (pas "motorbike")

preproc_path = "data_preproce_yolo"
out_path = "yolo_out"
os.makedirs(out_path, exist_ok=True)

images = sorted([f for f in os.listdir(preproc_path) if f.lower().endswith(".jpg")])

for img_name in images[:50]:
    img_path = os.path.join(preproc_path, img_name)
    img = cv2.imread(img_path)

    results = model(img)  # inference
    df = results.pandas().xyxy[0]  # xmin,ymin,xmax,ymax,confidence,class,name

    dfv = df[df['name'].isin(vehicle_classes)].copy()

    # Dessin boîtes
    for _, r in dfv.iterrows():
        x1, y1, x2, y2 = int(r.xmin), int(r.ymin), int(r.xmax), int(r.ymax)
        conf = float(r.confidence)
        label = f"{r['name']} {conf:.2f}"
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(img, label, (x1, max(15,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    cv2.imwrite(os.path.join(out_path, img_name), img)

print("Détection terminée, images annotées sauvegardées.")

Using cache found in /home/kawtar/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2026-2-9 Python-3.12.12 torch-2.10.0+cu128 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Détection terminée, images annotées sauvegardées.


# 3️⃣ Estimation de la congestion (OBJECTIF DU PROJET)

In [75]:
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# =========================
# 1️⃣ CONFIGURATION & SEUILS
# =========================
IMG_DIR = "dataSet/MVI_20012"
FPS = 25
LINE_Y = 350 

# Zone de la route (pour l'occupation) : [x, y, largeur, hauteur]
ROAD_ROI = [50, 200, 540, 300] 
ROAD_AREA = ROAD_ROI[2] * ROAD_ROI[3]

# Seuils
STOP_THRESHOLD = 1.5  # pixels : si bouge moins que ça, il est considéré immobile
STOP_FRAMES = 15      # nb de frames pour confirmer l'arrêt (file d'attente)

# YOLO
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.classes = [2, 3, 5, 7] # car, motorcycle, bus, truck

# =========================
# 2️⃣ VARIABLES DE SUIVI
# =========================
# Tracks persistants : {id: {'pos': (x,y), 'class': str, 'stopped': int, 'counted': bool}}
tracks = {}
count_by_class = defaultdict(int)

# Lister les images
images = sorted([f for f in os.listdir(IMG_DIR) if f.lower().endswith(".jpg")])
if len(images) == 0:
    raise ValueError("Aucune image trouvée !")

# Output vidéo
first_frame = cv2.imread(os.path.join(IMG_DIR, images[0]))
h, w = first_frame.shape[:2]
out_video = cv2.VideoWriter('smart_traffic.mp4', cv2.VideoWriter_fourcc(*'mp4v'), FPS, (w, h))

print("Analyse en cours... Appuie sur 'q' pour quitter")

# =========================
# 3️⃣ BOUCLE PRINCIPALE
# =========================
for img_name in images:
    frame = cv2.imread(os.path.join(IMG_DIR, img_name))
    if frame is None: continue

    # Détection YOLO
    results = model(frame)
    df = results.pandas().xyxy[0]

    matched_ids = set()
    occ_area = 0
    stopped_count = 0
    speeds = []

    # Pour chaque détection
    for _, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        cls_name = row['name']

        # Occupancy
        occ_area += (x2 - x1) * (y2 - y1)

        # --- TRACKING PERSISTANT ---
        track_id = None
        for tid, tdata in tracks.items():
            if np.hypot(cx - tdata['pos'][0], cy - tdata['pos'][1]) < 50:
                track_id = tid
                break

        if track_id is None:
            # Nouveau track
            track_id = max(tracks.keys(), default=0) + 1
            tracks[track_id] = {'pos': (cx, cy), 'class': cls_name, 'stopped': 0, 'counted': False}
        else:
            # Mise à jour position
            prev_pos = tracks[track_id]['pos']
            speed = np.hypot(cx - prev_pos[0], cy - prev_pos[1])
            speeds.append(speed)

            # Stop logique
            if speed < STOP_THRESHOLD:
                tracks[track_id]['stopped'] += 1
            else:
                tracks[track_id]['stopped'] = 0

            # Comptage ligne
            if not tracks[track_id]['counted'] and prev_pos[1] < LINE_Y <= cy:
                count_by_class[cls_name] += 1
                tracks[track_id]['counted'] = True

            tracks[track_id]['pos'] = (cx, cy)

        matched_ids.add(track_id)

        # Dessin du rectangle
        color = (0, 0, 255) if tracks[track_id]['stopped'] > STOP_FRAMES else (0, 255, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    # Nettoyage des tracks qui n'ont pas été détectés (optionnel)
    tracks = {tid: t for tid, t in tracks.items() if tid in matched_ids}

    # --- 4️⃣ ANALYSE CONGESTION ---
    occupancy = (occ_area / ROAD_AREA) * 100
    avg_speed = np.mean(speeds) if speeds else 0

    level = "LOW"
    level_color = (0, 255, 0)
    if occupancy > 15 or sum([t['stopped'] > STOP_FRAMES for t in tracks.values()]) > 3:
        level = "MEDIUM"
        level_color = (0, 255, 255)
    if occupancy > 35 or sum([t['stopped'] > STOP_FRAMES for t in tracks.values()]) > 6:
        level = "HIGH"
        level_color = (0, 0, 255)

    # --- 5️⃣ VISUALISATION ---
    cv2.rectangle(frame, (10, 10), (320, 180), (0,0,0), -1)
    cv2.putText(frame, f"CONGESTION: {level}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, level_color, 2)
    cv2.putText(frame, f"Occupancy: {occupancy:.1f}%", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)
    stopped_count = sum([t['stopped'] > STOP_FRAMES for t in tracks.values()])
    cv2.putText(frame, f"Stopped: {stopped_count}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)

    y_offset = 130
    for cls, val in count_by_class.items():
        cv2.putText(frame, f"{cls.upper()}: {val}", (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)
        y_offset += 20

    cv2.line(frame, (0, LINE_Y), (w, LINE_Y), (255, 0, 0), 2)
    out_video.write(frame)

    # Affichage optionnel
    cv2.imshow("Smart City Traffic", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

# =========================
# 6️⃣ FIN
# =========================
out_video.release()
cv2.destroyAllWindows()

# Sauvegarde résultats
with open("resultat_final.txt", "w") as f:
    total = sum(count_by_class.values())
    f.write(f"Nombre total de véhicules détectés : {total}\n")
    f.write(f"Détail par classe : {dict(count_by_class)}\n")

print("Analyse terminée !")
print(f"Total véhicules : {sum(count_by_class.values())}")
print(f"Vidéo sauvegardée : smart_traffic.mp4")


Using cache found in /home/kawtar/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2026-2-9 Python-3.12.12 torch-2.10.0+cu128 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
QObject::moveToThread: Current thread (0x4f1bfa40) is not the object's thread (0x558744f0).
Cannot move to target thread (0x4f1bfa40)

QObject::moveToThread: Current thread (0x4f1bfa40) is not the object's thread (0x558744f0).
Cannot move to target thread (0x4f1bfa40)

QObject::moveToThread: Current thread (0x4f1bfa40) is not the object's thread (0x558744f0).
Cannot move to target thread (0x4f1bfa40)

QObject::moveToThread: Current thread (0x4f1bfa40) is not the object's thread (0x558744f0).
Cannot move to target thread (0x4f1bfa40)

QObject::moveToThread: Current thread (0x4f1bfa40) is not the object's thread (0x558744f0).
Cannot move to target thread (0x4f1bfa40)

QObject::moveToThread: Current thread (0x4f1bfa40) is not the object's thread 

Analyse en cours... Appuie sur 'q' pour quitter
Analyse terminée !
Total véhicules : 55
Vidéo sauvegardée : smart_traffic.mp4
